In [6]:
from abc import ABC, abstractmethod

# Interface para as estratégias de extração
class EstrategiaExtracao(ABC):
    @abstractmethod
    def extrair_texto(self, nome_arquivo, texto_alvo):
        """Método abstrato para extrair o texto do PDF."""
        pass

    @abstractmethod
    def processar_dataframe(self, texto_extraido):
        """Método abstrato para processar o texto extraído em um DataFrame."""
        pass

In [22]:
import pandas as pd
import os


# Implementação para extração de dados da seguradora Alfa
class ExtracaoAlfa(EstrategiaExtracao):
    def extrair_texto(self, nome_arquivo, texto_alvo):
        """Método placeholder, não necessário para CSVs."""
        pass

    def extrair_num_extrato(self, df):
        """Extrai o número do recibo (extrato) da coluna 'nrrecibo'."""
        return df['nrrecibo'].iloc[0]

    def processar_dataframe(self, nome_arquivo):
        """Processa o arquivo CSV e formata o DataFrame conforme necessário."""
        df = pd.read_csv(nome_arquivo, sep=';')

        # Extração de campos específicos
        df['data'] = df['dtemissao'].str.split().str[0]  # Data extraída da coluna 'dtemissao'
        df['ramo'] = df['documento'].str.split('/').str[1]  # Ramo extraído da coluna 'documento'
        df['apolice'] = df['documento'].str.replace('/', '')  # Apólice extraída e formatada
        df['parcela'] = df['Pr'].str.split('/').str[0]  # Parcela extraída da coluna 'Pr'
        df['premio'] = df['vlpreliq']  # Prêmio líquido
        df['perc_comissao'] = '0,00'  # Comissão em percentual (fixo)
        df['comissao'] = df['vllancamento']  # Valor da comissão
        df['segurado'] = df['nmseg']  # Nome do segurado

        # Formatação das novas colunas
        df['Endosso'] = df['documento']
        df['Segurado'] = df['segurado']
        df['Apolice'] = df['apolice']
        df['Prêmio'] = df['premio']
        df['%'] = df['perc_comissao']
        df['Comissão'] = df['comissao']
        df['Ramo'] = df['ramo']
        df['data'] =' leitura_data_arquivo(nome_arquivo)'
        df['recibo'] = df['nrrecibo']

        colunas = ['Endosso', 'Apolice', 'Segurado', 'Ramo', 'Prêmio', '%', 'Comissão']
        df = df[colunas]
        # Converte todos os dados para string
        df = df.astype(str)

        return df




In [21]:
import pandas as pd

class ExtracaoMitsui(EstrategiaExtracao):
    def extrair_texto(self, nome_arquivo, texto_alvo):
        """Método abstrato implementado sem funcionalidade (placeholder)."""
        pass

    def extrair_num_extrato(self, mitsui):
        """Extrai o número do extrato a partir de uma célula contendo 'Extrato:'."""
        def extrair_valor(cell):
            if isinstance(cell, str) and 'Extrato: ' in cell:
                posicao_inicial = cell.find('Extrato: ') + len('Extrato: ')
                posicao_final = cell.find(' ', posicao_inicial)
                if posicao_final == -1:
                    posicao_final = len(cell)
                return cell[posicao_inicial:posicao_final].strip()
            return None

        num_extrato = mitsui.applymap(extrair_valor).stack().dropna().iloc[0]
        return num_extrato

    def processar_dataframe(self, nome_arquivo):
        """Processa o arquivo Excel e formata o DataFrame conforme necessário."""
        

        df = pd.read_excel(nome_arquivo)
        num_extrato = self.extrair_num_extrato(df)

        df = pd.read_excel(nome_arquivo, skiprows=6, dtype={'Apolice': str})

        # Filtra linhas e colunas necessárias
        df = df.dropna(subset=['Segurado', 'Apolice'])
        df = df.dropna(axis=1, how='all')
        df = df.drop(columns=['Natureza', 'Valor Comissão Antecipada (R$)', 'Valor Comissão (R$)', 'Data Baixa'])
        df = df.rename(columns={'Prêmio (R$)': 'Prêmio', 'Valor Total Comissão (R$)': 'Comissão'})

        #df.columns = ['Endosso', 'Apolice', 'Segurado', 'Ramo', 'Prêmio', '%', 'Comissão']
  

        df['Prêmio'] = df['Prêmio'].apply(lambda x: "{:.2f}".format(x).replace('.', ',').strip())
        df['%'] = df['%'].apply(lambda x: "{:.2f}".format(x).replace('.', ',').strip())
        df['Comissão'] = df['Comissão'].apply(lambda x: "{:.2f}".format(x).replace('.', ',').strip())
        df['Ramo'] = df['Ramo'].str.split('-').str[0].str.strip()
        df = df.astype(str)



        return df




Num extrato


In [44]:

import re
from pdfminer.high_level import extract_text



def leitura_data_arquivo(nom_arquivo):
    ano_atual = 2024
    # Usar expressão regular para encontrar a data no nome do arquivo
    match = re.search(r'(\d{2}-\d{2})', nom_arquivo)
    if match:
        # Extrair a data encontrada
        data_str = match.group(1)
        # Dividir a data pelo traço para obter dia e mês
        dia_mes_split = data_str.split('-')
        dia = dia_mes_split[0]
        mes = dia_mes_split[1]

        data_formatada = f"{dia}/{mes}/{ano_atual}"
        return data_formatada
    else:
        return "Data não encontrada no nome do arquivo"
    




def leitura_data_recibo(nome_arquivo):
    ano_atual = 2024
    
    # Usar expressão regular para encontrar a data no nome do arquivo
    match_data = re.search(r'(\d{2}-\d{2})', nome_arquivo)
    match_numero = re.search(r'\((\d+)\)', nome_arquivo)
    
    if match_data:
        # Extrair a data encontrada
        data_str = match_data.group(1)
        # Dividir a data pelo traço para obter dia e mês
        dia, mes = data_str.split('-')

        # Formatar a data
        data_formatada = f"{ano_atual}{mes}{dia}"

        # Se um número entre parênteses for encontrado, adicionar à data formatada
        if match_numero:
            numero = match_numero.group(1)
            data_formatada += numero

        return data_formatada
    else:
        return "Data não encontrada no nome do arquivo"


def extrai_recibo(inicio, nome_arquivo, fim="\n"):
    primeira_pagina = extract_text(nome_arquivo)
    texto_referencia = inicio

    posicao_inicial = primeira_pagina.find(texto_referencia)

    if posicao_inicial != -1:
        # Encontrar a posição final após o texto de referência
        posicao_final = primeira_pagina.find(fim, posicao_inicial)
        
        if posicao_final == -1:
            # Caso "\n\n" não seja encontrado, assumir que o texto vai até o final da string
            posicao_final = len(primeira_pagina)
        
        cod = primeira_pagina[posicao_inicial + len(texto_referencia):posicao_final].strip()
    else:
        cod = leitura_data_recibo(nome_arquivo)
    return cod


In [35]:

def extrai_recibo_pdf(inicio, nome_arquivo, fim="\n"):
    """Função para extrair recibo de arquivos PDF."""
    try:
        # Abrir o PDF e extrair o texto
        doc = fitz.open(nome_arquivo)
        primeira_pagina = doc[0].get_text("text")  # Extrai texto da primeira página
        doc.close()

        # Procurar a posição do texto de referência
        posicao_inicial = primeira_pagina.find(inicio)
        if posicao_inicial != -1:
            posicao_final = primeira_pagina.find(fim, posicao_inicial)
            if posicao_final == -1:
                posicao_final = len(primeira_pagina)
            return primeira_pagina[posicao_inicial + len(inicio):posicao_final].strip()
        else:
            print(f"Texto de referência '{inicio}' não encontrado no PDF.")
            return None
    except Exception as e:
        print(f"Erro ao processar o PDF {nome_arquivo}: {str(e)}")
        return None
import pandas as pd
import re

def extrai_recibo_excel(texto_alvo, padrao, nome_arquivo):
    """Função para extrair informações de um arquivo Excel com base em um padrão específico."""
    try:
        # Carregar o arquivo Excel
        df = pd.read_excel(nome_arquivo)

        # Procurar o texto de referência e o padrão em todas as colunas do DataFrame
        for col in df.columns:
            for idx, valor in df[col].astype(str).items():
                if texto_alvo in valor:  # Verifica se o texto de referência está na célula
                    # Verifica se a linha contém o padrão de 4 dígitos
                    linha_completa = ' '.join(df.iloc[idx].astype(str))
                    correspondencia = re.search(padrao, linha_completa)
                    if correspondencia:
                        return correspondencia.group()  # Retorna o valor encontrado com base no padrão
                        
        print(f"Texto '{texto_alvo}' seguido pelo padrão '{padrao}' não encontrado no Excel.")
        return None
    except Exception as e:
        print(f"Erro ao processar o Excel {nome_arquivo}: {str(e)}")
        return None


In [ ]:
caminho_base = r'C:\Users\igorferreiraaraujo\Documents\ETL_wiz_lg\ETL_wiz_lg\files'


nome_arquivo = os.path.join(caminho_base, 'MITSUI 25-09.xlsx')

In [ ]:

#leitura_data_recibo(nome_arquivo)
extrai_recibo_excel('Extrato: ',r"\d{4}",nome_arquivo)
#leitura_data_arquivo(nome_arquivo)



'1458'

In [ ]:
import pandas as pd
import re
import fitz  


def leitura_data_recibo(nome_arquivo):
    ano_atual = 2024
    
    # Usar expressão regular para encontrar a data no nome do arquivo
    match_data = re.search(r'(\d{2}-\d{2})', nome_arquivo)
    match_numero = re.search(r'\((\d+)\)', nome_arquivo)
    
    if match_data:
        # Extrair a data encontrada
        data_str = match_data.group(1)
        # Dividir a data pelo traço para obter dia e mês
        dia, mes = data_str.split('-')

        # Formatar a data
        data_formatada = f"{ano_atual}{mes}{dia}"

        # Se um número entre parênteses for encontrado, adicionar à data formatada
        if match_numero:
            numero = match_numero.group(1)
            data_formatada += numero

        return data_formatada
    else:
        return "Data não encontrada no nome do arquivo"


def extrai_recibo(texto_alvo, padrao, nome_arquivo):
    """Função para extrair informações de arquivos Excel ou PDF com base em um padrão específico."""
    try:
        # Verifica a extensão do arquivo
        if nome_arquivo.endswith(".xlsx") or nome_arquivo.endswith(".xls"):
            # Carregar o arquivo Excel
            df = pd.read_excel(nome_arquivo)

            # Procurar o texto de referência e o padrão em todas as colunas do DataFrame
            for col in df.columns:
                for idx, valor in df[col].astype(str).items():
                    if texto_alvo in valor:  # Verifica se o texto de referência está na célula
                        # Verifica se a linha contém o padrão de 4 dígitos
                        linha_completa = ' '.join(df.iloc[idx].astype(str))
                        correspondencia = re.search(padrao, linha_completa)
                        if correspondencia:
                            return correspondencia.group()  # Retorna o valor encontrado com base no padrão

        elif nome_arquivo.endswith(".pdf"):

            doc = fitz.open(nome_arquivo)
            texto_completo = ""


            for page_num in range(len(doc)):
                pagina = doc[page_num]
                texto_completo += pagina.get_text("text")


            doc.close()


            if texto_alvo in texto_completo:
               
                indice_texto = texto_completo.find(texto_alvo)
                texto_a_partir_do_alvo = texto_completo[indice_texto:]
                correspondencia = re.search(padrao, texto_a_partir_do_alvo)
                if correspondencia:
                    return correspondencia.group()  # Retorna o valor encontrado com base no padrão

        print(f"Texto '{texto_alvo}' seguido pelo padrão '{padrao}' não encontrado no arquivo.")
        return None

    except:
        return leitura_data_recibo(nome_arquivo)

In [ ]:
caminho_base = r'C:\Users\igorferreiraaraujo\Documents\ETL_wiz_lg\ETL_wiz_lg\files'


nome_arquivo = os.path.join(caminho_base, 'CARDIF 18-09.pdf')
extrai_recibo('No. Recibo\n',r"\d{7}",nome_arquivo)
#extrai_recibo('No. Recibo ',nome_arquivo, '\n')

'1017966'